# Seq2seq model - pitch-based
- [_Joplin_](http://kern.humdrum.org/search?s=t&keyword=Joplin) dataset is used as the training data for this model.
- A song is parsed into a sequence (containing one or multiple measures) of chords. 
    - Chords are represented in sequence of pitches, separated by '/', thus this model is called pitch-based; i.e., chords are represented as a sequence of one hot vectors that representing different pitches. 
    - In [chord-based model](Seq2seq model - Joplin.ipynb), each chord is represented as a single multi-hot vector.

In [1]:
import os
import util

## Part 1 - chord sequence to chord sequence
### Load data
- _Joplin_ dataset is used. 
- The data is parsed from kern score using <code>rchoraledata</code> tool in [_Humdrum Extras_](http://wiki.ccarh.org/wiki/Humdrum_Extras). See the [embedding notebook](Chord_Embeddings - Joplin.ipynb) for details.
- Filtering data
    - Only songs in 2/4 time signature are selected since most of the songs in this dataset are in 2/4.

In [2]:
music_type = 'joplin'
dataset = 'datasets/' + music_type

In [3]:
files = [f for f in os.listdir(dataset+'/dat/') if f.endswith('.dat')]
files = files.sort()

In [4]:
files = [f for f in os.listdir(dataset+'/krn/') if f.endswith('.krn')]
filtered_files = [] # only use song with time sig 2/4

for file in files[:]:
    basename = file[:-4]
    #! transpose -k c $dataset/krn/$basename\.krn | rchoraledata > $dataset/dat/$basename\.dat
    ret = ! beat -Ca $dataset/krn/$basename\.krn | beat -Ua  | extractx -s '$1-$'  | ridx -H | sortcount -p
    if ret[0] == '100\t2\t4': filtered_files.append(file)

In [5]:
filtered_files.sort()
filtered_files = [f[:-4] + '.dat' for f in filtered_files]

### Build dictionaries for pitch and time information
- Indexing different pitches and time durations.

In [6]:
# c_data: [song, measure, chord, pitch]
# t_data: [song, measure, dur]
t_data, c_data, t_val2idx, t_idx2val, c_val2idx, c_idx2val = (util.parse_dat_dir(dataset, filtered_files))

#### Add start, delimeter, and end tokens to the dictionaries
- <code>S</code> -  the start of a sequence
- <code>END</code> - the end of a sequence
- <code>/</code> - a delimeter that separates different chords
- <code>/</code> - padding

In [7]:
c_val2idx['<S>'] = len(c_val2idx)
c_idx2val[c_val2idx['<S>']] = '<S>'
c_val2idx['/'] = len(c_val2idx)
c_idx2val[c_val2idx['/']] = '/'
c_val2idx['<END>'] = len(c_val2idx)
c_idx2val[c_val2idx['<END>']] = '<END>'
c_val2idx['<PAD>'] = len(c_val2idx)
c_idx2val[c_val2idx['<PAD>']] = '<PAD>'

t_val2idx['<S>'] = len(t_val2idx)
t_idx2val[t_val2idx['<S>']] = '<S>'
t_val2idx['<END>'] = len(t_val2idx)
t_idx2val[t_val2idx['<END>']] = '<END>'

In [8]:
meas_per_seq = 2
encoder_input_data, decoder_input_data, decoder_target_data = util.gen_data_rec_pitch_based_emb(c_data, c_val2idx, meas_per_seq=meas_per_seq) #[song, measure, chord]

In [9]:
encoder_input_data_01, decoder_input_data_01, decoder_target_data_01 = util.gen_data_rec_pitch_based(c_data, c_val2idx, meas_per_seq=meas_per_seq) #[song, measure, chord]

### Generate input and target data that fits the model requirements
- The data is in the form of a list of 3D arrays.
    - Each entry in the list represents a song.
    - The 3 dimensions are for <code>[num_seq, max_seq_len, num_features]</code>.
    - Padded with zeros if <code>max_seq_len</code> is not met.

In [10]:
encoder_input_data = util.get_data_fitted_emb(encoder_input_data, c_val2idx)
decoder_input_data = util.get_data_fitted_emb(decoder_input_data, c_val2idx)
decoder_target_data = util.get_data_fitted_emb(decoder_target_data, c_val2idx)

In [11]:
encoder_input_data_01 = util.get_data_fitted(encoder_input_data_01)
decoder_input_data_01 = util.get_data_fitted(decoder_input_data_01)
decoder_target_data_01 = util.get_data_fitted(decoder_target_data_01)

In [12]:
decoder_target_data[0]

array([[ 13,  14,  15, ..., 142, 142, 142],
       [ 13,  22, 140, ..., 142, 142, 142],
       [ 29,  11, 140, ..., 142, 142, 142],
       ..., 
       [ 11,  34,  45, ..., 142, 142, 142],
       [ 23,  10,  32, ..., 142, 142, 142],
       [ 77,  33,  12, ..., 142, 142, 142]], dtype=int32)

In [13]:
# load emb
import numpy as np
emb_mat = np.load(dataset+'/emb/emb.npz')['emb']
# concatenate emb_mat with <S>, <END>, /
temp = np.ones((1, emb_mat.shape[1]))
temp[0, : temp.shape[1]//2] = 0
emb_mat = np.concatenate((emb_mat, np.zeros((1,emb_mat.shape[1]))), axis=0)
emb_mat = np.concatenate((emb_mat, temp), axis=0)
emb_mat = np.concatenate((emb_mat, np.ones((1,emb_mat.shape[1]))), axis=0)
emb_mat = np.concatenate((emb_mat, 1-temp), axis=0)

### Build the seq2seq model - predict next chords sequence from current chords sequence
- Referenced from a [Keras seq2seq tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).
- Have a look at [Keras](https://keras.io), which is a easy-to-use neural network API.

In [14]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

batch_size = 64  # Batch size for training.
iterations = 40  # Number of epochs to train for.
latent_dim = 256 

Using TensorFlow backend.


In [15]:
num_encoder_tokens = emb_mat.shape[1]
num_decoder_tokens = num_encoder_tokens
max_encoder_seq_length = None # different max-seq-len for different songs
max_decoder_seq_length = None

In [16]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
shared_emb = Embedding(len(c_val2idx), emb_mat.shape[1], weights=[emb_mat], trainable=True)
encoder_inputs_emb = shared_emb(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, ))
decoder_inputs_emb = shared_emb(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(c_val2idx), activation='softmax', input_shape=(None, ))
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
#model.reset_states()
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 80)     11440       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 345088      embedding_1[0][0]                
__________

### Start training

In [23]:
from keras.utils import plot_model
plot_model(model, to_file='graphs/with_emb_joplin_model.pdf')

In [24]:
batch_size = 64  # Batch size for training.
iterations = 55  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

for it in range(iterations):
    print('='*50)
    print('Iteration {}/{}'.format(it + 1, iterations))
    for i in range(len(c_data)):
        model.fit([encoder_input_data[i], decoder_input_data[i]], decoder_target_data_01[i],
              batch_size=None,
              epochs=1,
              validation_split=0.01)

Iteration 1/55
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 3s 32ms/step - loss: 3.0923 - val_loss: 1.4812
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 33ms/step - loss: 2.8818 - val_loss: 1.7887
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 2s 25ms/step - loss: 2.7553 - val_loss: 2.9049
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 23ms/step - loss: 2.9911 - val_loss: 1.8617
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 3s 29ms/step - loss: 2.3156 - val_loss: 1.4609
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 25ms/step - loss: 2.8065 - val_loss: 2.3297
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 2s 26ms/step - loss: 2.7394 - val_loss: 1.

86/86 [==============================] - 2s 29ms/step - loss: 2.1987 - val_loss: 1.2311
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 26ms/step - loss: 2.4936 - val_loss: 1.6846
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 25ms/step - loss: 2.1620 - val_loss: 1.7268
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 31ms/step - loss: 2.3642 - val_loss: 1.5304
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 44ms/step - loss: 1.6515 - val_loss: 0.8540
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 3s 33ms/step - loss: 2.3820 - val_loss: 1.5216
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 40ms/step - loss: 1.9095 - val_loss: 1.0083
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [======

89/89 [==============================] - 3s 33ms/step - loss: 1.6867 - val_loss: 1.0872
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 3s 35ms/step - loss: 1.7651 - val_loss: 1.4750
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 23ms/step - loss: 1.6834 - val_loss: 1.4683
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 28ms/step - loss: 1.6741 - val_loss: 1.0378
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 35ms/step - loss: 1.7380 - val_loss: 1.0960
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 3s 32ms/step - loss: 2.4014 - val_loss: 1.6728
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 3s 35ms/step - loss: 1.9450 - val_loss: 1.4043
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [======

Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 26ms/step - loss: 1.4798 - val_loss: 0.8367
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 26ms/step - loss: 1.3782 - val_loss: 0.7991
Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 3s 33ms/step - loss: 1.5402 - val_loss: 0.6622
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 2s 26ms/step - loss: 1.6829 - val_loss: 1.3163
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 26ms/step - loss: 1.4962 - val_loss: 1.3233
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 36ms/step - loss: 1.8757 - val_loss: 1.0779
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 29ms/step - loss: 1.4391 - val_loss: 0.8350
Train on 91 s

81/81 [==============================] - 2s 27ms/step - loss: 1.5037 - val_loss: 0.8940
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 2s 26ms/step - loss: 1.3034 - val_loss: 1.0722
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 2s 27ms/step - loss: 1.4709 - val_loss: 0.9184
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 3s 29ms/step - loss: 1.4680 - val_loss: 1.0629
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 34ms/step - loss: 1.4918 - val_loss: 0.8911
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 3s 24ms/step - loss: 1.5443 - val_loss: 1.4151
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 27ms/step - loss: 1.2712 - val_loss: 1.0219
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

...

Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 2s 29ms/step - loss: 0.2141 - val_loss: 0.1961
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 2s 24ms/step - loss: 0.1611 - val_loss: 0.7429
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 22ms/step - loss: 0.1521 - val_loss: 1.1240
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 34ms/step - loss: 0.2584 - val_loss: 0.5561
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 29ms/step - loss: 0.2242 - val_loss: 0.4015
Train on 91 samples, validate on 1 samples
Epoch 1/1
91/91 [==============================] - 2s 24ms/step - loss: 0.2682 - val_loss: 0.4586
Iteration 49/55
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 3s 27ms/step - loss: 0.2209 - val_loss: 0.17

81/81 [==============================] - 2s 27ms/step - loss: 0.1791 - val_loss: 0.5569
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 3s 26ms/step - loss: 0.1711 - val_loss: 1.4336
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 30ms/step - loss: 0.2208 - val_loss: 0.9637
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 2s 22ms/step - loss: 0.2189 - val_loss: 1.2421
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 26ms/step - loss: 0.1851 - val_loss: 0.2798
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 27ms/step - loss: 0.2467 - val_loss: 0.3834
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 25ms/step - loss: 0.2074 - val_loss: 0.6670
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

97/97 [==============================] - 3s 28ms/step - loss: 0.1985 - val_loss: 0.1341
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 28ms/step - loss: 0.2835 - val_loss: 0.7764
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 0.1753 - val_loss: 0.8796
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 23ms/step - loss: 0.1578 - val_loss: 0.4973
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 2s 28ms/step - loss: 0.2131 - val_loss: 0.8758
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 24ms/step - loss: 0.1575 - val_loss: 0.8118
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 2s 25ms/step - loss: 0.1656 - val_loss: 0.4547
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [===

Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 25ms/step - loss: 0.1857 - val_loss: 0.9616
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 2s 25ms/step - loss: 0.1953 - val_loss: 1.1523
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 30ms/step - loss: 0.2488 - val_loss: 0.7621
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 44ms/step - loss: 0.3283 - val_loss: 0.4498
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 2s 28ms/step - loss: 0.1742 - val_loss: 0.7795
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 33ms/step - loss: 0.2241 - val_loss: 0.6675
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 23ms/step - loss: 0.1456 - val_loss: 0.3803
Train on 84 s

99/99 [==============================] - 3s 28ms/step - loss: 0.1277 - val_loss: 0.3746
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 19ms/step - loss: 0.0955 - val_loss: 0.9962
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 2s 28ms/step - loss: 0.0995 - val_loss: 0.2426
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 32ms/step - loss: 0.1972 - val_loss: 0.8288
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 2s 28ms/step - loss: 0.1918 - val_loss: 0.8349
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 3s 30ms/step - loss: 0.1889 - val_loss: 0.4733
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 2s 26ms/step - loss: 0.1546 - val_loss: 0.4484
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [======

In [25]:
save_name = 'models/with_emb_joplin_' + str(meas_per_seq) + '.h5'
model.save(save_name)

/anaconda3/envs/nlu/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


### Sampling the notes
- Different sampling methods are used.
    - $\arg\max$: selecting the pitch with highest probability.
        - Deterministic.
        - Potential problem: always choose the most frequent values, the output tends to be repetitive.
    - Temperature sampling with different $\tau$s.
        - Randomly choose based on temperature sampling; stochastic.
        - Higher $\tau$s adds variety to the music generated.
        - Potential problem: large $\tau$ increases the chance of inharmony.
- Sampling methods can be seen at [sampling.py](sampling.py).

In [167]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs_emb, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

print([decoder_inputs] + decoder_states_inputs)
print([decoder_outputs] + decoder_states)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = c_idx2val
reverse_target_char_index = c_idx2val
input_token_index = c_val2idx
target_token_index = c_val2idx

[<tf.Tensor 'input_2:0' shape=(?, ?) dtype=float32>, <tf.Tensor 'input_15:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'input_16:0' shape=(?, 256) dtype=float32>]
[<tf.Tensor 'dense_1_7/truediv:0' shape=(?, ?, 143) dtype=float32>, <tf.Tensor 'lstm_2_7/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_2_7/while/Exit_3:0' shape=(?, 256) dtype=float32>]


In [89]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 80)          11440     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 345088    
Total params: 356,528
Trainable params: 356,528
Non-trainable params: 0
_________________________________________________________________


In [90]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 80)     11440       input_2[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_2 (LS

In [45]:
import sampling

In [188]:
def decode_sequence_new(input_seq, samp_fun=sampling.argmax, max_decoder_seq_length=500):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1), dtype='int32')
    target_seq[0, 0] = c_val2idx['<S>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    target_ret = []
    target_ret.append(target_seq)
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq, states_value[0], states_value[1]])
        
        
        # Sample a token
        sampled_token_index = samp_fun(output_tokens[0, -1, :]) # sampling using argmax
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += str(sampled_char) + ' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        target_ret.append(target_seq[0,0])

        # Update states
        states_value = [h, c]

    return decoded_sentence, np.array([np.array(target_ret, dtype='int32')])

In [197]:
def print_seq(input_seq, display_name=True):

    naming_func = lambda x: x
    if display_name:
        naming_func = util.get_note_name_by_idx

    sent = ''
    
    for idx in input_seq[0,:] :
        if idx == len(c_val2idx) - 1: break
        sampled_char = reverse_target_char_index[idx]
        if sampled_char == '<S>' or sampled_char == '<END>' or sampled_char == '/': sent += sampled_char
        elif sampled_char >= 0:
            sent += str(naming_func(sampled_char))
        else:
            sent += '-' + str(naming_func(np.abs(sampled_char)))
        sent += ' '
    
    return sent

In [198]:
target_token_index = c_val2idx

chords_gen_argmax = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/emb/argmax/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure

    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence_new(input_seq)
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence_new(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_argmax.append(curr_gen)

In [200]:
chords_gen_tau1 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/emb/tau1.0/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=1.0))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=1.0))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau1.append(curr_gen)

In [201]:
chords_gen_tau2 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/emb/tau0.9/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=0.9))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=0.9))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau2.append(curr_gen)

In [202]:
chords_gen_tau3 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/emb/tau0.8/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=0.8))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence_new(input_seq, samp_fun=sampling.temperature(tau=0.8))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau3.append(curr_gen)

## Part 2 - chord sequence to time sequence
### Load data
- _Joplin_ dataset is used. 
- The data is parsed from kern score using <code>rchoraledata</code> tool in [_Humdrum Extras_](http://wiki.ccarh.org/wiki/Humdrum_Extras). See the [embedding notebook](Chord_Embeddings - Joplin.ipynb) for details.
- Filtering data
    - Only songs in 2/4 time signature are selected since most of the songs in this dataset are in 2/4.

In [205]:
_, decoder_input_data, decoder_target_data = util.gen_data_rec_pitch_based_times(c_data, c_val2idx, t_data, t_val2idx, meas_per_seq=meas_per_seq) #[song, measure, chord]

In [206]:
decoder_input_data = util.get_data_fitted(decoder_input_data)
decoder_target_data = util.get_data_fitted(decoder_target_data)

In [209]:
num_decoder_tokens = len(t_val2idx)

In [210]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
shared_emb = Embedding(len(c_val2idx), emb_mat.shape[1], weights=[emb_mat], trainable=True)
encoder_inputs_emb = shared_emb(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [211]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [212]:
model.reset_states()
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [213]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [214]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 80)     11440       input_17[0][0]                   
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, None, 12)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 345088      embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_4 (LS

In [215]:
batch_size = 64  # Batch size for training.
iterations = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

for it in range(iterations):
    print('='*50)
    print('Iteration {}/{}'.format(it + 1, iterations))
    for i in range(len(c_data)):
        model.fit([encoder_input_data[i], decoder_input_data[i]], decoder_target_data[i],
              batch_size=None,
              epochs=1,
              validation_split=0.01)

Iteration 1/50
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 2s 22ms/step - loss: 1.2779 - val_loss: 1.3980
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 14ms/step - loss: 0.6767 - val_loss: 0.6762
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 1s 13ms/step - loss: 0.6573 - val_loss: 0.7192
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 16ms/step - loss: 0.6189 - val_loss: 0.5379
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 14ms/step - loss: 0.6333 - val_loss: 0.5637
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 13ms/step - loss: 0.4936 - val_loss: 1.0107
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 15ms/step - loss: 0.5887 - val_loss: 0.

86/86 [==============================] - 1s 13ms/step - loss: 0.5695 - val_loss: 0.4427
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 14ms/step - loss: 0.4447 - val_loss: 0.8373
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 12ms/step - loss: 0.5262 - val_loss: 0.4526
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 0.3819 - val_loss: 1.0254
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 22ms/step - loss: 0.4665 - val_loss: 0.7489
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 2s 23ms/step - loss: 0.4441 - val_loss: 0.6618
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 3s 31ms/step - loss: 2.4520 - val_loss: 0.2985
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [======

89/89 [==============================] - 1s 13ms/step - loss: 0.4640 - val_loss: 0.4555
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 1s 15ms/step - loss: 0.3665 - val_loss: 0.4333
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 11ms/step - loss: 0.7577 - val_loss: 0.3600
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 15ms/step - loss: 0.4338 - val_loss: 0.5288
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 17ms/step - loss: 0.4810 - val_loss: 0.4203
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 2s 19ms/step - loss: 0.3832 - val_loss: 0.8108
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 18ms/step - loss: 0.5197 - val_loss: 0.4248
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [======

Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 12ms/step - loss: 0.5562 - val_loss: 0.3093
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 12ms/step - loss: 0.5206 - val_loss: 0.2516
Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 1s 14ms/step - loss: 0.3820 - val_loss: 0.2565
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 1s 12ms/step - loss: 0.2914 - val_loss: 0.2519
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 11ms/step - loss: 0.4448 - val_loss: 0.5666
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 17ms/step - loss: 0.3691 - val_loss: 0.6307
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 0.4017 - val_loss: 0.4095
Train on 91 s

81/81 [==============================] - 1s 14ms/step - loss: 0.3160 - val_loss: 0.4837
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 12ms/step - loss: 0.4291 - val_loss: 0.3303
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 15ms/step - loss: 0.3425 - val_loss: 0.2700
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 2s 18ms/step - loss: 0.3626 - val_loss: 0.3383
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 19ms/step - loss: 0.3049 - val_loss: 0.4102
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 1s 13ms/step - loss: 0.5831 - val_loss: 0.2103
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 14ms/step - loss: 0.3694 - val_loss: 0.3614
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

...

73/73 [==============================] - 1s 14ms/step - loss: 0.1569 - val_loss: 0.1434
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 1s 13ms/step - loss: 0.1249 - val_loss: 0.1680
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 0.0963 - val_loss: 0.4393
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 14ms/step - loss: 0.1758 - val_loss: 1.7740
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 0.1014 - val_loss: 0.1274
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 12ms/step - loss: 0.1252 - val_loss: 0.3345
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 1s 14ms/step - loss: 0.0914 - val_loss: 0.1733
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [===

Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 12ms/step - loss: 0.1146 - val_loss: 0.1522
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 14ms/step - loss: 0.0904 - val_loss: 0.7617
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 20ms/step - loss: 0.1002 - val_loss: 0.4794
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 14ms/step - loss: 0.1091 - val_loss: 0.4937
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 18ms/step - loss: 0.1692 - val_loss: 0.1704
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 12ms/step - loss: 0.0850 - val_loss: 0.5317
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 12ms/step - loss: 0.1308 - val_loss: 0.0623
Train on 83 s

85/85 [==============================] - 1s 12ms/step - loss: 0.0995 - val_loss: 0.0717
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 14ms/step - loss: 0.0830 - val_loss: 0.1213
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 0.1429 - val_loss: 0.3226
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 15ms/step - loss: 0.0915 - val_loss: 0.7572
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 18ms/step - loss: 0.1150 - val_loss: 0.6092
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 18ms/step - loss: 0.0788 - val_loss: 0.3992
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 13ms/step - loss: 0.1158 - val_loss: 0.0549
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [======

Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 1s 17ms/step - loss: 0.0838 - val_loss: 0.3436
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 1s 13ms/step - loss: 0.0421 - val_loss: 0.1734
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 0.1011 - val_loss: 0.0774
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 17ms/step - loss: 0.0963 - val_loss: 0.5234
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 0.1197 - val_loss: 0.0918
Train on 91 samples, validate on 1 samples
Epoch 1/1
91/91 [==============================] - 1s 13ms/step - loss: 0.0937 - val_loss: 0.2897
Iteration 49/50
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 1s 14ms/step - loss: 0.0836 - val_loss: 0.19

81/81 [==============================] - 1s 14ms/step - loss: 0.0719 - val_loss: 0.4926
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 1s 14ms/step - loss: 0.1024 - val_loss: 0.6156
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 16ms/step - loss: 0.0897 - val_loss: 0.2854
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 1s 11ms/step - loss: 0.1145 - val_loss: 0.0881
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 13ms/step - loss: 0.0887 - val_loss: 0.0539
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 13ms/step - loss: 0.0960 - val_loss: 0.7988
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 12ms/step - loss: 0.0830 - val_loss: 0.2133
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

In [216]:
save_name = 'models/with_emb_joplin_times' + str(meas_per_seq) + '.h5'
model.save(save_name)

/anaconda3/envs/nlu/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [218]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = c_idx2val
reverse_target_char_index = t_idx2val
input_token_index = c_val2idx
target_token_index = t_val2idx

In [221]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 80)          11440     
_________________________________________________________________
lstm_3 (LSTM)                [(None, 256), (None, 256) 345088    
Total params: 356,528
Trainable params: 356,528
Non-trainable params: 0
_________________________________________________________________


In [222]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, None, 12)     0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  275456      input_18[0][0]                   
                                                                 input_19[0][0]                   
          

In [260]:
def decode_sequence_new(input_seq, samp_fun=sampling.argmax, max_decoder_seq_length=500):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1, num_decoder_tokens), dtype='int32')
    target_seq[0, 0, t_val2idx['<S>']] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    target_ret = []
    target_ret.append(target_seq)
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq, states_value[0], states_value[1]])
        
        
        # Sample a token
        sampled_token_index = samp_fun(output_tokens[0, -1, :]) # sampling using argmax
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += str(sampled_char) + ' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens), dtype='int32')
        target_seq[0, 0, sampled_token_index] = 1
        target_ret.append(target_seq[0,0,:])

        # Update states
        states_value = [h, c]

    return decoded_sentence, np.array([target_ret])

In [269]:
def print_seq(input_seq, display_name=True):
    
    naming_func = lambda x: x
    if display_name:
        naming_func = util.get_note_name_by_idx

    sent = ''
    
    for measure in input_seq[0] :
        indices = np.where(measure > 0)[0]
        if len(indices != 0):
            idx = indices[0]
            sampled_char = reverse_target_char_index[idx]
            if sampled_char == '<S>' or sampled_char == '<END>' or sampled_char == '/': sent += sampled_char
            elif sampled_char >= 0:
                sent += str(naming_func(sampled_char))
            else:
                sent += '-' + str(naming_func(np.abs(sampled_char)))
            sent += ' '
        else: break
    
    return sent

In [271]:
# generate from argmax chords
for i, song in enumerate(chords_gen_argmax): # which song
    
    filename = 'output/joplin/times/emb/argmax/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')
    
    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence_new(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [272]:
reverse_target_char_index = t_idx2val
# generate from tau1.0 chords
for i, song in enumerate(chords_gen_tau1): # which song
    
    filename = 'output/joplin/times/emb/tau1.0/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [273]:
# generate from tau0.9 chords
for i, song in enumerate(chords_gen_tau2): # which song
    
    filename = 'output/joplin/times/emb/tau0.9/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [274]:
# generate from tau0.8 chords
for i, song in enumerate(chords_gen_tau3): # which song
    
    filename = 'output/joplin/times/emb/tau0.8/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()